In [2]:
import os
import pickle as pkl
import random
import time

import fasttext
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from fastembed import TextEmbedding
from tqdm import tqdm
from xgboost import XGBClassifier

from prompt_classifier.modeling.dspy_llm import LlmClassifier
from prompt_classifier.modeling.fasttext import FastTextClassifier
from prompt_classifier.modeling.nli_modernbert import ModernBERTNLI
from prompt_classifier.metrics import evaluate

load_dotenv()
random.seed(1)

In [5]:
# Load Jigsaw dataset
jigsaw_splits = {'train': 'train_dataset.csv', 'validation': 'val_dataset.csv', 'test': 'test_dataset.csv'}
jigsaw_df = pd.read_csv("hf://datasets/Arsive/toxicity_classification_jigsaw/" + jigsaw_splits["validation"])

jigsaw_df = jigsaw_df[(jigsaw_df["toxic"] == 1) |
                            (jigsaw_df["severe_toxic"] == 1) |
                            (jigsaw_df["obscene"] == 1) |
                            (jigsaw_df["threat"] == 1) |
                            (jigsaw_df["insult"] == 1) |
                            (jigsaw_df["identity_hate"] == 1)]

jigsaw_df = jigsaw_df.rename(columns={"comment_text": "messages"})
jigsaw_df["label"] = 0
jigsaw_df = jigsaw_df.dropna(subset=["messages"])
jigsaw_df = jigsaw_df[["messages", "label"]]

# Load OLID dataset
olid_splits = {'train': 'train.csv', 'test': 'test.csv'}
olid_df = pd.read_csv("hf://datasets/christophsonntag/OLID/" + olid_splits["train"])

olid_df = olid_df.rename(columns={"cleaned_tweet": "messages"})
olid_df["label"] = 0
olid_df = olid_df.dropna(subset=["messages"])
olid_df = olid_df[["messages", "label"]]

# Load hateXplain dataset
hateXplain = pd.read_parquet("hf://datasets/nirmalendu01/hateXplain_filtered/data/train-00000-of-00001.parquet")
hateXplain = hateXplain.rename(columns={"test_case": "messages"})
hateXplain = hateXplain[(hateXplain["gold_label"] == "hateful")]
hateXplain = hateXplain.rename(columns={"gold_label": "label"})
hateXplain = hateXplain[["messages", "label"]]
hateXplain = hateXplain.dropna(subset=["messages"])

# Load TUKE Slovak dataset
tuke_sk_splits = {'train': 'train.json', 'test': 'test.json'}
tuke_sk_df = pd.read_json("hf://datasets/TUKE-KEMT/hate_speech_slovak/" + tuke_sk_splits["train"], lines=True)
tuke_sk_df = tuke_sk_df.rename(columns={"text": "messages"})
tuke_sk_df = tuke_sk_df[tuke_sk_df["label"] == 0]
tuke_sk_df = tuke_sk_df[["messages", "label"]]
tuke_sk_df = tuke_sk_df.dropna(subset=["messages"])

datasets = {
    'jigsaw': jigsaw_df,
    'olid': olid_df,
    'hate_xplain': hateXplain,
    'tuke_sk': tuke_sk_df
}

In [ ]:
baai_embedding = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    providers=["CUDAExecutionProvider"]
)

mini_embedding = TextEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    providers=["CUDAExecutionProvider"],
)

# TF-IDF
tfidf_finance = pkl.load(open("./models/tfidf_finance.pkl", "rb"))
tfidf_healthcare = pkl.load(open("./models/tfidf_healthcare.pkl", "rb"))
tfidf_law = pkl.load(open("./models/tfidf_law.pkl", "rb"))

2025-04-03 22:52:04.037329509 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-04-03 22:52:04.037387066 [W:onnxruntime:, session_state.cc:1170 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2025-04-03 22:52:04.793673176 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-04-03 22:52:04.793713046 [W:onnxruntime:, session_state.cc:1170 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:
# LLM Classifier -> zmenit na qwen2.5
model_name = "Qwen2.5:7b"

for domain, inference_df in datasets:
    try:
        llm_classifier_finance = LlmClassifier(
            api_key=os.getenv("OPENAI_API_KEY"),
            api_base=os.getenv("PROXY_URL"),
            model_name=model_name,
            domain="finance",
            train_data=inference_df,
            test_data=inference_df,
        )

        llm_classifier_healthcare = LlmClassifier(
            api_key=os.getenv("OPENAI_API_KEY"),
            api_base=os.getenv("PROXY_URL"),
            model_name=model_name,
            domain="healthcare",
            train_data=inference_df,
            test_data=inference_df,
        )

        llm_classifier_law = LlmClassifier(
            api_key=os.getenv("OPENAI_API_KEY"),
            api_base=os.getenv("PROXY_URL"),
            model_name=model_name,
            domain="law",
            train_data=inference_df,
            test_data=inference_df,
        )

        # Load models
        llm_classifier_finance.load_model("models/gpt-4o-mini-finance.json")
        llm_classifier_healthcare.load_model("models/gpt-4o-mini-healthcare.json")
        llm_classifier_law.load_model("models/gpt-4o-mini-law.json")

        predictions_llm = []
        prediction_times_llm = []
        actuals_llm = []

        # Get predictions for each prompt
        for _, row in tqdm(inference_df.head(1000).iterrows(), total=1000):
            start_time = time.perf_counter_ns()

            # Get predictions from all models
            pred_finance = llm_classifier_finance.predict_single(row["prompt"])
            pred_healthcare = llm_classifier_healthcare.predict_single(row["prompt"])
            pred_law = llm_classifier_law.predict_single(row["prompt"])

            end_time = time.perf_counter_ns()
            prediction_times_llm.append(end_time - start_time)

            # If any model predicts 1, final prediction is 0
            predictions_llm.append(0 if (pred_finance == 1 or pred_healthcare == 1 or pred_law == 1) else 1)
            actuals_llm.append(row["label"])
        
        evaluate(predictions=predictions_llm, true_labels=actuals_llm, latency=prediction_times_llm, domain=domain, embed_model="ada-002", model_name=model_name, train_acc=0.0, cost=0.0)
    except Exception as e:
        print(f"Error running GPT model: {e}")

In [ ]:
bert_classifier_finance = ModernBERTNLI(domain="finance")
bert_classifier_healthcare = ModernBERTNLI(domain="healthcare")
bert_classifier_law = ModernBERTNLI(domain="law")

predictions_bert = []
prediction_times_bert = []
actuals_bert = []

try:
    # Move models to GPU
    bert_classifier_finance.classifier.model.to("cuda")
    bert_classifier_healthcare.classifier.model.to("cuda")
    bert_classifier_law.classifier.model.to("cuda")

    for inference_df in datasets:
        # Get predictions for each prompt
        for _, row in tqdm(inference_df.iterrows(), total=len(inference_df)):
            start_time = time.perf_counter_ns()

            # Get predictions from all models
            pred_finance = bert_classifier_finance.predict(row["prompt"])
            pred_healthcare = bert_classifier_healthcare.predict(row["prompt"])
            pred_law = bert_classifier_law.predict(row["prompt"])

            end_time = time.perf_counter_ns()
            prediction_times_bert.append(end_time - start_time)

            # If any model predicts 1, final prediction is 0
            predictions_bert.append(0 if (pred_finance == 1 or pred_healthcare == 1 or pred_law == 1) else 1)
            actuals_bert.append(row["label"])

    evaluate(predictions=predictions_bert, true_labels=actuals_bert, latency=prediction_times_bert, domain=domain, embed_model="BERT", model_name="ModernBERT", train_acc=0.0, cost=0.0)
except Exception as e:
    print(f"Error running ModernBERT models: {e}")

In [ ]:
actuals_ft = []
predictions_ft = []
prediction_times_ft = []

# fastText
for inference_df in datasets:
    fasttext_classifier_finance = FastTextClassifier(train_data=inference_df, test_data=inference_df)
    fasttext_classifier_finance.model = fasttext.load_model("models/fastText_finance_fasttext.bin")

    fasttext_classifier_healthcare = FastTextClassifier(train_data=inference_df, test_data=inference_df)
    fasttext_classifier_healthcare.model = fasttext.load_model("models/fastText_healthcare_fasttext.bin")

    fasttext_classifier_law = FastTextClassifier(train_data=inference_df, test_data=inference_df)
    fasttext_classifier_law.model = fasttext.load_model("models/fastText_law_fasttext.bin")

    for _, row in tqdm(inference_df.iterrows(), total=len(inference_df)):
        text = str(row["prompt"])
        query = text.replace("\n", "")

        start_time = time.perf_counter_ns()

        # Predictions from all three classifiers
        prediction_finance = fasttext_classifier_finance.model.predict(query)
        prediction_healthcare = fasttext_classifier_healthcare.model.predict(query)
        prediction_law = fasttext_classifier_law.model.predict(query)

        end_time = time.perf_counter_ns()
        prediction_times_ft.append(end_time - start_time)

        predictions_ft.append(0 if (prediction_finance[0][0] == "__label__1" or prediction_healthcare[0][0] == "__label__1" or prediction_law[0][0] == "__label__1") else 1)
        actuals_ft.append(row["label"])

        evaluate(predictions=predictions_ft, true_labels=actuals_ft, latency=prediction_times_ft, domain=domain, embed_model="fastText", model_name="fastText", train_acc=0.0, cost=0.0)

In [ ]:
# Embedding test data
for inference_df in datasets:
    start_time = time.perf_counter_ns()
    test_embeds = np.array(list(baai_embedding.embed(inference_df["prompt"])))
    end_time = time.perf_counter_ns()
    embed_times = end_time - start_time

    mean_embed_time = embed_times / len(inference_df)

    with open("models/SVM_finance_baai.pkl", "rb") as svm_file:
        svm_classifier_finance = pkl.load(svm_file)

    with open("models/SVM_healthcare_baai.pkl", "rb") as svm_file:
        svm_classifier_healthcare = pkl.load(svm_file)

    with open("models/SVM_law_baai.pkl", "rb") as svm_file:
        svm_classifier_law = pkl.load(svm_file)

    xgb_classifier_finance = XGBClassifier()
    xgb_classifier_healthcare = XGBClassifier()
    xgb_classifier_law = XGBClassifier()

    xgb_classifier_finance.load_model("models/XGBoost_finance_baai.json")
    xgb_classifier_healthcare.load_model("models/XGBoost_healthcare_baai.json")
    xgb_classifier_law.load_model("models/XGBoost_law_baai.json")

    predictions_xgb = []
    predictions_svm = []

    prediction_times_xgb = []
    prediction_times_svm = []

    actuals_ml = []

    for test_embed in test_embeds:
        # SVM predictions
        start_time = time.perf_counter_ns()
        pred_finance = svm_classifier_finance.predict(test_embed.reshape(1, -1))
        pred_healthcare = svm_classifier_healthcare.predict(test_embed.reshape(1, -1))
        pred_law = svm_classifier_law.predict(test_embed.reshape(1, -1))
        end_time = time.perf_counter_ns()

        prediction_times_svm.append(end_time - start_time)
        # If any model predicts 1, final prediction is 0
        predictions_svm.append(0 if (pred_finance[0] == 1 or pred_healthcare[0] == 1 or pred_law[0] == 1) else 1)

        # XGBoost predictions
        start_time = time.perf_counter_ns()
        pred_finance = xgb_classifier_finance.predict(test_embed.reshape(1, -1))
        pred_healthcare = xgb_classifier_healthcare.predict(test_embed.reshape(1, -1))
        pred_law = xgb_classifier_law.predict(test_embed.reshape(1, -1))
        end_time = time.perf_counter_ns()

        prediction_times_xgb.append(end_time - start_time)
        # If any model predicts 1, final prediction is 0
        predictions_xgb.append(0 if (pred_finance[0] == 1 or pred_healthcare[0] == 1 or pred_law[0] == 1) else 1)
        actuals_ml.append(row["label"])

    
    evaluate(predictions=predictions_svm, true_labels=actuals_ml, latency=prediction_times_svm, domain=domain, embed_model="", model_name="fastText", train_acc=0.0, cost=0.0)
    evaluate(predictions=predictions_xgb, true_labels=actuals_ml, latency=prediction_times_xgb, domain=domain, embed_model="", model_name="XGBoost", train_acc=0.0, cost=0.0)